# Preprocessing

1. 한글&숫자 남기고 제거
2. hanspell
3. mecab

### Library

In [1]:
# !git clone https://github.com/ssut/py-hanspell.git

In [38]:
base_path = '/home/hyejeongeun/chatbot/Data/Base/'
save_path = '/home/hyejeongeun/chatbot/Data/QnA/'

In [2]:
# 수정한 hanspell 불러오기
from py_hanspell.hanspell import spell_checker
from py_hanspell.hanspell.constants import CheckResult

In [4]:
# !pip install kss

In [3]:
import numpy as np
import pandas as pd
import kss
import re
from tqdm.notebook import tqdm
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load Data

In [4]:
df = pd.read_csv(base_path+'qna_table.csv')

In [7]:
df[:5]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3080121,False,False,상품,어떤 유리인가요?,2021-03-19T01:16:01.000+09:00,None,None
1,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 화이트,3079369,True,False,배송,배송상태가 배송완료로되어있는데 아직상품이안왔습니다. 언제쯤도착하는지 확인부탁드릴게요.,2021-03-18T22:31:54.000+09:00,None,None
2,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 블랙,3078560,True,False,교환,상부 유리의 모서리가 다 깨져있습니다...재배송 부탁드려요,2021-03-18T20:13:11.000+09:00,None,None
3,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00
4,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3076525,True,False,상품,배송 언제 되는걸까요...?,2021-03-18T15:31:16.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n유선상 연락 및 안내드렸습니다.\r\n...,2021-03-18T15:52:19.000+09:00


### 비밀글 & 답변 없음 제거

In [5]:
# 전체 개수
df.shape

(125543, 11)

In [6]:
# 둘 다 비밀글이 아닌 것
# 즉, 둘 중 하나라도 비밀글이면 drop
df_remove = df[(df['question']!='비밀글입니다.')&(df['answer']!='비밀글입니다.')]

In [7]:
df_remove.shape

(95340, 11)

In [8]:
# 답변 없음 제거
df_remove_ = df_remove[df_remove['answer']!='None']

In [9]:
df_remove_.shape

(95305, 11)

In [13]:
df_remove_[:5]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
3,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00
4,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3076525,True,False,상품,배송 언제 되는걸까요...?,2021-03-18T15:31:16.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n유선상 연락 및 안내드렸습니다.\r\n...,2021-03-18T15:52:19.000+09:00
5,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 화이트,3076317,True,False,배송,구매 하였는데 배송 메모는 무시해주세요ㅠ 다른곳에서 산거 수정을 못했어요ㅠㅠ 빠른배...,2021-03-18T15:05:32.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n네 ~~주문확인되는데로 최대한 빠른 배...,2021-03-18T16:08:01.000+09:00
8,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 블랙,3073853,True,False,상품,조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요.. 육각 나사 렌...,2021-03-18T10:23:56.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n따로 발송해드리도록 하겠습니다.\r\n...,2021-03-18T15:26:29.000+09:00
9,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3070524,False,False,배송,주문해서 다음주 월요일(3/22)에 받고싶은데..언제 주문하면될까요?,2021-03-17T19:50:58.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n월요일 수령은 장담드리기 어렵습니다.주말...,2021-03-18T16:45:11.000+09:00


### 내용 중복 확인 (수정 필요, 아직 처리 X)

In [14]:
len(df_remove_['question'].unique())

92984

In [15]:
len(df_remove_['answer'].unique())

80645

In [10]:
tmp_drop = df_remove_.drop_duplicates(['question','answer'], keep = 'first')
tmp_drop.reset_index(inplace=True, drop=True)
tmp_drop[:5]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00
1,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3076525,True,False,상품,배송 언제 되는걸까요...?,2021-03-18T15:31:16.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n유선상 연락 및 안내드렸습니다.\r\n...,2021-03-18T15:52:19.000+09:00
2,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 화이트,3076317,True,False,배송,구매 하였는데 배송 메모는 무시해주세요ㅠ 다른곳에서 산거 수정을 못했어요ㅠㅠ 빠른배...,2021-03-18T15:05:32.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n네 ~~주문확인되는데로 최대한 빠른 배...,2021-03-18T16:08:01.000+09:00
3,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 블랙,3073853,True,False,상품,조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요.. 육각 나사 렌...,2021-03-18T10:23:56.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n따로 발송해드리도록 하겠습니다.\r\n...,2021-03-18T15:26:29.000+09:00
4,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3070524,False,False,배송,주문해서 다음주 월요일(3/22)에 받고싶은데..언제 주문하면될까요?,2021-03-17T19:50:58.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n월요일 수령은 장담드리기 어렵습니다.주말...,2021-03-18T16:45:11.000+09:00


In [11]:
tmp_drop.shape

(95190, 11)

In [12]:
# 개수 확인하기
count = tmp_drop[['question','answer']].value_counts()
len(count[count.values==1])

95190

In [19]:
# tmp_drop

## 전처리

In [13]:
df = tmp_drop

In [14]:
que = df['question'].to_numpy()
ans = df['answer'].to_numpy()

### 한글&숫자만 남기고 제거

In [15]:
def text_cleaning(texts): # 수정완료
    # 한국어를 제외한 글자를 제거하는 패턴.
    corpus = []
    for i in range(0, len(texts)):
      sentences = re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", str(texts[i])) #특수문자를 제거하는 패턴
      sentences = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 |0-9|]", " ", sentences)
      sentences = re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ', sentences) # 자음, 모음만 있는 경우

      sentences = re.sub(r"^\s+", '', sentences) #remove space from start
      sentences = re.sub(r'\s+$', '', sentences) #remove space from the end
      
      sentences = re.sub(r'\s+', ' ', sentences) #remove extra space

      corpus.append(sentences)
    return corpus

In [16]:
pre_que = text_cleaning(que)
pre_ans = text_cleaning(ans)

In [18]:
len(pre_que), len(pre_ans)

(95190, 95190)

In [17]:
# df에 업데이트
df['question_clean'] = pre_que
df['answer_clean'] = pre_ans

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### hanspell 맞춤법

In [30]:
# que
def hanspell_(texts):
    spell_check = []
    for line in tqdm(texts):
        spell = spell_checker.check(line).checked
        spell_check.append(spell)
    return spell_check

spell_check_question = hanspell_(pre_que)

  0%|          | 0/95190 [00:00<?, ?it/s]

In [32]:
spell_check_question[:5]

['<None>',
 '배송 언제 되는 걸까요',
 '구매하였는데 배송 메모는 무시해주세요 다른 곳에서 산거 수정을 못했어요 빠른 배송 부탁드립니다',
 '조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요 육각 나사 렌치 보내주실 수 있나요',
 '주문해서 다음 주 월요일 3 22에 받고 싶은데 언제 주문하면 될까요']

In [33]:
df['question_spellcheck'] = spell_check_question

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
# ans
def hanspell_(texts):
    spell_check = []
    for line in tqdm(texts):
        spell = spell_checker.check(line).checked
        spell_check.append(spell)
    return spell_check

spell_check_answer = hanspell_(pre_ans)

  0%|          | 0/95190 [00:00<?, ?it/s]

In [41]:
spell_check_answer[:5]

['안녕하세요 고객님 먼데이 하우스입니다 불편을 드려 죄송합니다 유선상 연락을 드렸으나 부재중이시네요 안타깝게도 조립을 시작하셨으면 반품이 어렵습니다 사이트 내 하자 사진과 함께 교환 신청을 해주시면 하자인 부분만 재출고하고 있습니다 감사합니다 3',
 '안녕하세요 고객님 먼데이 하우스입니다 유선상 연락 및 안내드렸습니다 감사합니다 1',
 '안녕하세요 고객님 먼데이 하우스입니다 네 주문 확인되는 데로 최대한 빠른 배송 위해 최선을 다하겠습니다 감사합니다 1',
 '안녕하세요 고객님 먼데이 하우스입니다 따로 발송해드리도록 하겠습니다 감사합니다 1',
 '안녕하세요 고객님 먼데이 하우스입니다 월요일 수령은 장담 드리기 어렵습니다 주말 휴무이기 때문에 금요일에 출고해드리면 토요일에 받을 수 있어요 구매 후 다음날 정도 출고 진행하며 배송업체 스케줄에 따라 1 3일 정도 소요되는 점 참고 바랍니다 감사합니다 2']

In [42]:
df['answer_spellcheck'] = spell_check_answer

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.columns

In [43]:
df.to_csv(save_path+'qna_table_hanspell.csv', index=False)